In [1]:
import random
import healpy as hp
import glob
from scipy.optimize import curve_fit
import pickle
from importlib import reload
import time
import scipy
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import pylab
from pylab import arange, show, cm
from astropy import units as uq
import gc
from astropy.io import fits

### Specific qubic modules
from qubicpack.utilities import Qubic_DataDir
from pysimulators import FitsArray
import pysm3
import pysm3.units as u
import pysm3.utils as utils
import qubic
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from qubic import NamasterLib as nam
from qubic import mcmc
from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

### FGBuster functions module
from fgbuster import get_instrument, get_sky, get_observation, ilc, basic_comp_sep, harmonic_ilc, weighted_comp_sep, multi_res_comp_sep, harmonic_ilc_alm  # Predefined instrumental and sky-creation configurations
from fgbuster.visualization import corner_norm, plot_component
from fgbuster.mixingmatrix import MixingMatrix
from fgbuster.observation_helpers import _rj2cmb, _jysr2rj, get_noise_realization

# Imports needed for component separation
from fgbuster import (separation_recipes, xForecast, CMB, Dust, Synchrotron, FreeFree, PowerLaw,  # sky-fitting model
                      basic_comp_sep)  # separation routine

import ComponentSeparation


### Some initializations, to be replaced with specific path, or to modify in bash
os.environ['QUBIC_DATADIR'] = '/home/mathias/Bureau/qubic/qubic'
os.environ['QUBIC_DICT'] = '/home/mathias/Bureau/qubic/qubic/dicts'
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

### Qubic dictionaries for 150GHz and 220Ghz
config_150, config_220 = 'FI-150', 'FI-220'
dictfilename150 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_150)
dictfilename220 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_220)
d150, d220 = qubic.qubicdict.qubicDict(), qubic.qubicdict.qubicDict()
d150.read_from_file(dictfilename150)
d220.read_from_file(dictfilename220)
qub_dic = {'150': d150, '220': d220}
center = qubic.equ2gal(d220['RA_center'], d220['DEC_center'])

### Read some stuff


# Read dictionary chosen
d150['focal_length'] = 0.3
d150['nside'] = 256

#Define the number of reconstruction bands:
nbands = 3
d150['nf_recon'] = nbands
d150['nf_sub'] = nbands

# Read dictionary chosen
d220['focal_length'] = 0.3
d220['nside'] = 256

#Define the number of reconstruction bands:
nbands = 3
d220['nf_recon'] = nbands
d220['nf_sub'] = nbands

npix = 12 * d150['nside'] ** 2
Nf = int(d150['nf_sub'])
band = d150['filter_nu'] / 1e9
filter_relative_bandwidth = d150['filter_relative_bandwidth']
npix = 12 * d220['nside'] ** 2
Nf = int(d220['nf_sub'])
band = d220['filter_nu'] / 1e9
filter_relative_bandwidth = d220['filter_relative_bandwidth']
a, nus_edge, nus_in, df, e, Nbbands_in = qubic.compute_freq(band, Nf, filter_relative_bandwidth)

def give_me_freqs_fwhm(dic, Nb) :
    band = dic['filter_nu'] / 1e9
    filter_relative_bandwidth = dic['filter_relative_bandwidth']
    a, nus_edge, nus_in, df, e, Nbbands_in = qubic.compute_freq(band, Nb, filter_relative_bandwidth)
    return nus_in, dic['synthbeam_peak150_fwhm'] * 150 / nus_in


def open_picklefile(directory, name, var) :

    """
    Open a pickle file from saved data
    """

    tab = pickle.load(open(directory + name, 'rb'))
    variable = tab[var]
    return variable

cov150 = open_picklefile('/home/mathias/Bureau/FG-Buster/', 'Coverage_dtheta_15_pointing_3000_256.pkl', 'coverage')


def give_me_mask_apo(okpix = None, lmin = 2, lmax = 512, delta_ell = 16) :
    mask = np.zeros(12 * d150['nside']**2)
    mask[okpix] = 1
    # Namaster object
    Namaster = nam.Namaster(mask, lmin=2, lmax=512, delta_ell=16)
    apo = Namaster.mask_apo
    return apo

# Define okpix
okpix_inside = (cov150 > (0.5*np.max(cov150)))
okpix = (cov150 > (0.1*np.max(cov150)))

apo = give_me_mask_apo(okpix = okpix)

In [2]:
def plot(map_in, fwhm, fwhm_target, i_f, istk) :
    
    map_out, _, _ = ComponentSeparation.same_resol(map_in, fwhm, fwhm_target)
    
    plt.figure(figsize = (16, 10))
    hp.gnomview(map_in[i_f, istk, :], rot = center, reso = 15, sub = (1, 2, 1), cmap = 'jet')
    hp.gnomview(map_out[i_f, istk, :], rot = center, reso = 15, sub = (1, 2, 2), cmap = 'jet')
    plt.show()

nsim1 = 0
nsim2 = 1
ite = 1

##########################################
##########################################

noise = [4e3]
Nside = 256
eps = 1e-8

##########################################
##########################################

beta_150 = np.zeros((len(noise), nsim2 - nsim1))
beta_220 = np.zeros((len(noise), nsim2 - nsim1))
beta_combined = np.zeros((len(noise), nsim2 - nsim1))
T_150 = np.zeros((len(noise), nsim2 - nsim1))
T_220 = np.zeros((len(noise), nsim2 - nsim1))
T_combined = np.zeros((len(noise), nsim2 - nsim1))
nu0 = 150.

for i, j in enumerate(noise) :
    for k in range(nsim1, nsim2) :
        print(i, j, k)
        dir = '/home/mathias/Bureau/FG-Buster/pickle_file_fgbuster/Maps+noise/'
        #all_maps = np.zeros(((2*nbands, 3, 12*256**2)))
        #all_N = np.zeros(((2*nbands, 3, 12*256**2)))
        maps_150 = open_picklefile(dir + '150GHz/3bands/whitenoise/{}years/'.format(int(j)), 'maps_150_noise_3bands_spa_nu_corr_False_nyears_{}_{}.pkl'.format(int(j), k+1), 'maps')
        #maps_220 = open_picklefile(dir + '220GHz/3bands/whitenoise/{}years/'.format(int(j)), 'maps_220_noise_3bands_spa_nu_corr_False_nyears_{}_{}.pkl'.format(int(j), k+1), 'maps')

        N_150 = open_picklefile(dir + '150GHz/3bands/whitenoise/{}years/'.format(int(j)), 'maps_150_noise_3bands_spa_nu_corr_False_nyears_{}_{}.pkl'.format(int(j), k+1), 'noise')
        #N_220 = open_picklefile(dir + '220GHz/3bands/whitenoise/{}years/'.format(int(j)), 'maps_220_noise_3bands_spa_nu_corr_False_nyears_{}_{}.pkl'.format(int(j), k+1), 'noise')

        freqs, fwhmdegs = give_me_freqs_fwhm(d150, nbands)
        fwhm_final = np.max(fwhmdegs) + eps
        
        print("From {} to {}".format(np.max(fwhmdegs), fwhm_final))
        
        #plot(maps_150, fwhmdegs, fwhm_final, 0, 0)
        #plot(maps_150, fwhmdegs, fwhm_final, 0, 1)
        #plot(maps_150, fwhmdegs, fwhm_final, 0, 2)
        
        
        map_in = np.zeros(((nbands, 3, 12*Nside**2)))
        N_in = np.zeros(((nbands, 3, 12*Nside**2)))
        for i in range(nbands) :
            map_in[i] = hp.pixelfunc.ud_grade(maps_150[i], Nside)
            N_in[i] = hp.pixelfunc.ud_grade(N_150[i], Nside)
        cov_in = hp.pixelfunc.ud_grade(cov150, Nside)
        okpix_inside = (cov_in > (0.5*np.max(cov_in)))
        
        #plot(map_in, fwhmdegs, fwhm_final, 0, 1)

        R_150 = ComponentSeparation.CompSep(d150).fg_buster(map_in, N_in, [CMB(), Dust(nu0, temp = 20.)], freq=freqs, fwhmdeg=fwhmdegs, target = fwhm_final, okpix = okpix_inside, Stokesparameter = 'IQU')
        
        #freqs, fwhmdegs = give_me_freqs_fwhm(d220, nbands)
        #fwhm_final = np.max(fwhmdegs) + 1e-8

        #R_220 = ComponentSeparation.CompSep(d220).fg_buster(maps_220, N_220, [CMB(), Dust(nu0, temp = 20.)], freq=freqs, fwhmdeg=fwhmdegs, target = fwhm_final, okpix = okpix_inside)

        #all_maps[:nbands] = maps_150
        #all_maps[nbands:] = maps_220
        #all_N[:nbands] = N_150
        #all_N[nbands:] = N_220

        freqs150, fwhmdegs150 = give_me_freqs_fwhm(d150, nbands)
        #freqs220, fwhmdegs220 = give_me_freqs_fwhm(d220, nbands)

        #freqs = list(freqs150) + list(freqs220)
        #FWHM = list(fwhmdegs150) + list(fwhmdegs220)

        #target_fwhm = np.max(FWHM) + 1e-8

        #R = ComponentSeparation.CompSep(d220).fg_buster(all_maps, all_N, [CMB(), Dust(nu0, temp = 20.)], freq=freqs, fwhmdeg=FWHM, target = target_fwhm, okpix = okpix_inside)

        print(" \n ========= Estimation of beta ========== \n ")

        print(R_150.x[0])
        #print(R_220.x[0])
        #print(R.x[0])

        #print()

        #beta_150[i, k - nsim1] = R_150.x[0]
        #beta_220[i, k - nsim1] = R_220.x[0]
        #beta_combined[i, k - nsim1] = R.x[0]

        del maps_150
        del R_150

0 4000.0 0
From 0.42999269332029166 to 0.42999270332029166


/home/mathias/Bureau/FG-Buster/ComponentSeparation.py:127: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ins.frequency = freq
/home/mathias/Bureau/FG-Buster/ComponentSeparation.py:128: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ins.fwhm = fwhmdeg
/home/mathias/Bureau/FG-Buster/ComponentSeparation.py:130: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ins.depth_i = give_me_rms_I(map_N[:, :, okpix], self.nside)
/home/mathias/Bureau/FG-Buster/ComponentSeparation.py:131: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  ins.

[0.28818458 0.23306826 0.20171126]
[0.40609903 0.32895859 0.28313216]
 
 ========= Estimation of beta ========== 
 
1.5603263199777906
